In [1]:
import pandas as pd
import numpy as np

import os

def load_hotel_reserve():
  customer_tb = pd.read_csv('../awesomebook-master/data/customer.csv')
  hotel_tb = pd.read_csv('../awesomebook-master/data/hotel.csv')
  reserve_tb = pd.read_csv('../awesomebook-master/data/reserve.csv')
  return customer_tb, hotel_tb, reserve_tb


def load_holiday_mst():
  holiday_tb = pd.read_csv('../awesomebook-master/data/holiday_mst.csv',
                           index_col=False)
  return holiday_tb


def load_production():
  production_tb = pd.read_csv('../awesomebook-master/data/production.csv')
  return production_tb


def load_production_missing_num():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_num.csv')
  return production_tb


def load_production_missing_category():
  production_tb = pd.read_csv('../awesomebook-master/data/production_missing_category.csv')
  return production_tb


def load_monthly_index():
  monthly_index_tb = \
    pd.read_csv('../awesomebook-master/data/monthly_index.csv')
  return monthly_index_tb


def load_meros_txt():
  with open('../awesomebook-master/data/txt/meros.txt', 'r') as f:
    meros = f.read()
    f.close()
  return meros


In [2]:
customer_tb, hotel_tb, reserve_tb = load_hotel_reserve()

print(reserve_tb)

     reserve_id hotel_id customer_id     reserve_datetime checkin_date  \
0            r1     h_75         c_1  2016-03-06 13:09:42   2016-03-26   
1            r2    h_219         c_1  2016-07-16 23:39:55   2016-07-20   
2            r3    h_179         c_1  2016-09-24 10:03:17   2016-10-19   
3            r4    h_214         c_1  2017-03-08 03:20:10   2017-03-29   
4            r5     h_16         c_1  2017-09-05 19:50:37   2017-09-22   
...         ...      ...         ...                  ...          ...   
4025      r4026    h_129       c_999  2017-06-27 23:00:02   2017-07-10   
4026      r4027     h_97       c_999  2017-09-29 05:24:57   2017-10-09   
4027      r4028     h_27       c_999  2018-03-14 05:01:45   2018-04-02   
4028      r4029     h_48      c_1000  2016-04-16 15:20:17   2016-05-10   
4029      r4030    h_117      c_1000  2016-06-06 08:16:51   2016-07-06   

     checkin_time checkout_date  people_num  total_price  
0        10:00:00    2016-03-29           4        9

In [5]:
result = reserve_tb.groupby('hotel_id') \
                   .agg({'reserve_id': 'count', 'customer_id': 'nunique'})

result

,reserve_id,customer_id
hotel_id,,
h_1,10,10
h_10,3,3
h_100,20,19
h_101,17,17
h_102,13,13
...,...,...
h_95,13,13
h_96,13,13
h_97,16,16


In [6]:
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']

result

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13
...,...,...,...
295,h_95,13,13
296,h_96,13,13
297,h_97,16,16
298,h_98,17,16


In [8]:
result = reserve_tb.groupby(['hotel_id', 'people_num'])['total_price'] \
                   .sum().reset_index()

result

,hotel_id,people_num,total_price
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000


In [9]:
result.rename(columns={'total_price': 'price_sum'}, inplace=True)

result

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200
...,...,...,...
1154,h_98,3,793800
1155,h_98,4,453600
1156,h_99,1,179200
1157,h_99,2,448000


In [ ]:
result = reserve_tb.groupby('hotel_id') \
                   .agg({'total_price': ['max', 'min', 'mean', 'median', lambda x: np.percentile(x, q=20)]}) \
                   .reset_index()

result.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean', 'price_median', 'price_20per']

result

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400.0,73080.0
1,h_10,67200,11200,42933.333333,50400.0,26880.0
2,h_100,57600,4800,27600.000000,28800.0,9600.0
3,h_101,168000,14000,75764.705882,56000.0,30800.0
4,h_102,72000,12000,32769.230769,24000.0,18000.0
...,...,...,...,...,...,...
295,h_95,518400,43200,275815.384615,259200.0,146880.0
296,h_96,66600,7400,33015.384615,29600.0,17760.0
297,h_97,250800,20900,83600.000000,62700.0,20900.0
298,h_98,226800,18900,96723.529412,75600.0,56700.0


In [11]:
result = reserve_tb.groupby('hotel_id') \
                   .agg({'total_price': ['var', 'std']}) \
                   .reset_index()

result.columns = ['hotel_id', 'price_var', 'price_std']

result.fillna(0, inplace=True)

result

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159
...,...,...,...
295,h_95,3.313772e+10,182037.696857
296,h_96,3.159231e+08,17774.225072
297,h_97,5.474685e+09,73991.116584
298,h_98,3.432893e+09,58590.896578


In [13]:
reserve_tb['total_price'].round(-3).mode(3)

0    10000
1    20000
2    40000
Name: total_price, dtype: int64

In [15]:
reserve_tb['reserve_datetime'] = pd.to_datetime(
    reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S'
)

reserve_tb['log_no'] = reserve_tb.groupby('customer_id')['reserve_datetime'] \
                                 .rank(ascending=True, method='first')

reserve_tb

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0
...,...,...,...,...,...,...,...,...,...,...
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,4.0
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,5.0
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,6.0
4028,r4029,h_48,c_1000,2016-04-16 15:20:17,2016-05-10,09:30:00,2016-05-13,4,540000,1.0


In [16]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']

rsv_cnt_tb

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13
...,...,...
295,h_95,13
296,h_96,13
297,h_97,16
298,h_98,17


In [17]:
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'].rank(ascending=False, method='min')

rsv_cnt_tb

,hotel_id,rsv_cnt,rsv_cnt_rank
0,h_1,10,235.0
1,h_10,3,300.0
2,h_100,20,12.0
3,h_101,17,43.0
4,h_102,13,139.0
...,...,...,...
295,h_95,13,139.0
296,h_96,13,139.0
297,h_97,16,60.0
298,h_98,17,43.0


In [ ]:
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)

rsv_cnt_tb